In [ ]:
z-score 정규화시키는 코드 작성하기. 확인해보기
배치정규화 ok
드랍아웃 ok
셔플 ok
learning_rate확인
교차검증 
데이터 다시 나누기.

## 이미지파일 numpy배열로 변환해서 dataset 만들기

In [ ]:
from PIL import Image
import numpy as np
import glob

path_dir = "D:/OCT-Image/make_dataset/train"

categories = ['CNV', 'DME', 'DRUSEN', 'NORMAL']
nb_classes = len(categories)
image_w = 299
image_h = 299

X = []
Y = []

for idx, categorie in enumerate(categories):  # 라벨링작업
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    image_dir = path_dir + "/"
    files = glob.glob(image_dir + categorie + "/*")  # 경로에 대응되는 모든파일
        
    for i, file in enumerate(files):
        print(file)
        img = Image.open(file)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        label = np.asarray(label)
        X.append(data)
        Y.append(label)
                               
X_data = np.array(X)
Y_data = np.array(Y)
np.save("X_data.npy", X_data)
np.save("Y_data.npy", Y_data)

In [ ]:
print(X_data.shape)
print(Y_data.shape)

In [9]:
from PIL import Image

file = "D:/OCT-Image/make_dataset/train/CNV/CNV-13823-1.jpeg"
img = Image.open(file)
img = img.convert("RGB")
img = img.resize((299, 299))
data = np.asarray(img)

In [17]:
int(np.array(['1','2','3']))

TypeError: only size-1 arrays can be converted to Python scalars

In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam
import numpy as np
import time

def z_score_normalization(input_maen, input_std, data):
    result = (data - input_mean) / input_std
    return result

X_data = np.load('X_data.npy')
Y_data = np.load('Y_data.npy')

input_mean = 128
input_std = 128

X_data = StandardScaler().fit_transform(X_data)

X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.1, random_state=777)


# model 구성
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=(299,299,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same", activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))
print(model.summary())


# model 학습과정설정
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
              metrics=['acc'])

print("start train")
start = time.time() # 현재시간 측정
history = model.fit(x=X_train, y=Y_train, batch_size=128, epochs=100)
print("time :", (time.time() - start) / 60) # 학습하는데 걸리는 시간(현재시간 - 아까 측정한 시간)
model.save('Model_1.h5') # 모델 저장시키기

MemoryError: Unable to allocate 167. GiB for an array with shape (83484, 299, 299, 3) and data type float64

In [ ]:
# acc 그래프
plt.figure("train_graph")
plt.plot(history.history['acc'], 'y', label='train_acc')
plt.plot(history.history['loss'], 'b', label='train_loss')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()